# Tiling Utilities

This notebook contains a demo of the tiling utilities included in `IceFloeTracker.jl`. In particular, the following workflows are illustrated:

- Getting tiling with tiles of a given size
- Getting the optimal tile size given an initial tile side length

Run the cell below to set up the computation environment.

In [ ]:
HOME = "../.." # path to the root of the project two levels up

# Activate the environment
using Pkg
Pkg.activate(HOME)
Pkg.precompile()

using IceFloeTracker: get_tiles, get_tile_dims, load
using ColorTypes: RGBA

#### Load the image

In [ ]:
imgpath = "test/test_inputs/NE_Greenland_truecolor.2020162.aqua.250m.tiff"
img = load(joinpath(HOME,imgpath))

#### Estimate tile size

Say we want to split the image into roughly 8x8 tiles.

In [ ]:

prelim_sizes = size(img) .÷ 8

#### Choosing tile size and building tilings

The `get_tiles` function builds a tiling given an image (array) and a tile side length.



In [ ]:
# Let's view the documentation for the function
@info @doc get_tiles

In [ ]:
tiles = get_tiles(img, prelim_sizes[1] + 1) # deliberately using a size that is too large

In [ ]:
# Top left tile dimensions
get_tile_dims(tiles[1])

Note that the bottom right tile has been extended to cover the rest of the image as a uniform tiling of side length `prelim_sizes[1] + 1` fails to cover the full image.

In [ ]:
# View the first tile
img[tiles[1]...]

Further, note that the tiles on the right and bottom edges are slightly bigger.

In [ ]:
# Bottom right tile dimensions
get_tile_dims(tiles[end])

### View the full tiling

In [ ]:
function build_tiling_illustration(img, side_length)

    # Create new canvas to draw on
    newimg = similar(img, RGBA{Float64})

    # Apply transparency to the tiles
    for tile_coords in get_tiles(img, side_length)
        tile = @view img[tile_coords...]
        alpha = rand(0.5:0.05:1)
        transparent_tile = map(c -> RGBA(c.r, c.g, c.b, alpha), tile)
        newimg[tile_coords...] .= transparent_tile
    end

    # View the image
    newimg
end

build_tiling_illustration(img, prelim_sizes[1] + 1)


### Optimal tile side length

Perhaps there is a fitter tiling that is close to the originally desired tiling. We can use the `get_optimal_tile_size` function to determine whether this is possible.

In [ ]:
using IceFloeTracker: get_optimal_tile_size

@info @doc get_optimal_tile_size

In [ ]:
best_side_length = get_optimal_tile_size(prelim_sizes[1] + 1, size(img))

As expected, a tile size of 1016 is not optimal for this image. The function `get_optimal_tile_size` suggests a fitter tiling is possible using tiles of 1015 pixels in side length for this image.

In [ ]:
build_tiling_illustration(img, best_side_length)

In [ ]:
# Area of corner tile with optimal side length
get_tile_dims(get_tiles(img, best_side_length)[end]) |> prod

Note that the area of the corner tile for the suboptimal tiling is larger.

In [ ]:
get_tile_dims(get_tiles(img, prelim_sizes[1] + 1)[end]) |> prod

The proportion of pixels missed by a uniform tiling can be computed with `get_area_missed`.

In [ ]:
using IceFloeTracker: get_area_missed
areamissed_optimal = get_area_missed(best_side_length, size(img))
@show areamissed_optimal
@assert get_area_missed(prelim_sizes[1] + 1, size(img)) > get_area_missed(best_side_length, size(img))